# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [1]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'CapstoneProjectHyperdrive'

experiment=Experiment(ws, experiment_name)

In [2]:
# Import the data
from azureml.core.dataset import Dataset
import pandas as pd


key = 'TicTacToe data'
if key in ws.datasets.keys():
    dataset = ws.datasets[key]
    print("Dataset found")
    datastore = ws.get_default_datastore()
    train = Dataset.Tabular.from_delimited_files([(datastore,'trn.csv')])

else:
    datastore = ws.get_default_datastore()
    datastore.upload_files(files = ['./trn.csv'])
    train = Dataset.Tabular.from_delimited_files([(datastore,'trn.csv')])
    train = train.register(ws,key)

Dataset found


# Create Compute cluster

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


cpu_cluster_name = "compute-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Creating a new compute cluster...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

# Can poll for a minimum number of nodes and for a specific timeout. 
# If no min node count is provided it uses the scale settings for the cluster.
compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration
Dataset contains categorical data do tree based models will work nice. So I chose xgboost for the better performance.

Parameters: Learning rate is common parameter in tuning ML models. I used n_estimators to control overfitting. As it is a tree based algorithm we should consider tree depth while tuning the model.

I used Bandit policy for early stopping using a slack factor of 0.15.

Randomparametersampler is used to sample my hyperparameters. As my parameters are descrete, I can use choice method to sample the parameter randomly. This method is fast well suited for descrete dataset. 



In [4]:
from azureml.core import ScriptRunConfig
from azureml.core import Environment
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
import os
import shutil

In [5]:
ds_file = Dataset.File.from_files(path = [(datastore, ('trn.csv'))])

In [6]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

#env_deploy=Environment.get(workspace=ws, name='AzureML-Tutorial')

sklearn_env = Environment.from_conda_specification(name="sklearn-env",file_path="conda_dependencies.yaml")
conda_dep = CondaDependencies()

# Installs numpy version 1.17.0 conda package
conda_dep.add_conda_package("xgboost==0.90")

# Adds dependencies to PythonSection of myenv
sklearn_env.python.conda_dependencies=conda_dep

In [7]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.15, evaluation_interval=1, delay_evaluation=10)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling( {
    'lrate': choice(0.1,0.5,1.0,5.0),
    'nest': choice(80,100,120),
    'depth':choice(1,2,3)
    })

# create environment
#curated_env_name = 'AzureML-TensorFlow-2.2-GPU'
#tf_env = Environment.get(workspace=ws, name=curated_env_name)

#
#tf_env.python.conda_dependencies.add_pip_package('joblib')


# configure and submit your training run
run_config = ScriptRunConfig(source_directory='.',
                            script='train.py',
                            compute_target=compute_target,
                            arguments=['--path', ds_file.as_named_input('input').as_mount()],
                            environment=sklearn_env)

hyperdrive_run_config = HyperDriveConfig(run_config=run_config,
                                     hyperparameter_sampling=param_sampling, 
                                     policy=early_termination_policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=12,
                                     max_concurrent_runs=4)



In [8]:
#TODO: Submit your experiment
run = experiment.submit(config=hyperdrive_run_config)

In [9]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [10]:
best_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Parameter arguements:',parameter_values['runDefinition']['arguments'])

Best Run Id:  HD_54245eac-5eaf-4025-9ef6-5198aba85cca_11

 Accuracy: 0.9861111111111112

 Parameter arguements: ['--path', 'DatasetConsumptionConfig:input', '--depth', '3', '--lrate', '1', '--nest', '100']


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [67]:
from azureml.core.webservice import AciWebservice, LocalWebservice
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.model import Model
import requests
import json

In [94]:
config = AciWebservice.deploy_configuration(cpu_cores=1, 
                                            memory_gb=4, 
                                            enable_app_insights=True,
                                            description='bestmodel')

model = Model(ws, 'bestmodel')


# Using the same environment for deployment
#env_deploy=Environment.get(workspace=ws, name='AzureML-Tutorial')
#env_deploy = Environment.get(workspace=ws, name='AzureML-AutoML')

Inference_config = InferenceConfig(entry_script="score.py", environment=sklearn_env)


# Deploy the best model
service = Model.deploy(workspace=ws, 
                       name='bestmodel', 
                       models=[model], 
                       inference_config=Inference_config, 
                       deployment_config=config)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [95]:
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-04-05 13:09:10+00:00 Creating Container Registry if not exists.
2021-04-05 13:09:11+00:00 Registering the environment.
2021-04-05 13:09:12+00:00 Use the existing image.
2021-04-05 13:09:12+00:00 Generating deployment configuration.
2021-04-05 13:09:12+00:00 Submitting deployment to compute.
2021-04-05 13:09:15+00:00 Checking the status of deployment bestmodel.

TODO: In the cell below, print the logs of the web service and delete the service

In [91]:
print(service.get_logs())

None
